In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys,os
sys.path.append('..')
from backtester import matlab, backtester
from backtester.analysis import *
from backtester.strategy import StrategyBase, OptParam, OptParamArray
from backtester.swarms.ranking import SwarmRanker
from backtester.swarms.rebalancing import SwarmRebalance
from backtester.swarms.filters import SwarmFilter
from backtester.costs import CostsManagerEXOFixed
from backtester.exoinfo import EXOInfo
from backtester.swarms.rankingclasses import *
from backtester.swarms.swarm import Swarm

import statsmodels.tsa.api as smt

import pandas as pd
import numpy as np
import scipy

In [ ]:
from scripts.settings import *
try:
    from scripts.settings_local import *
except:
    pass

from exobuilder.data.exostorage import EXOStorage

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)


exo_filter = '*'     # All 
#exo_filter = 'ES_'  # ES only
exo_filter = ''  # ES Collars (incl Bearish, Bullish, BW and vanilla)

[print(exo) for exo in storage.exo_list(exo_filter)];

In [ ]:
class Strategy_RollingHurstExp(StrategyBase):
    name = 'Strategy_RollingHurstExp'


    def __init__(self, strategy_context):
        # Initialize parent class
        super().__init__(strategy_context)
   
    

    def calc_entryexit_rules(self, hurst_window, hurst_value, hurst_median_window, rules_index):
        def hurst(ts):
            """Returns the Hurst Exponent of the time series vector ts"""
            # Create the range of lag values
            lags = range(2, 20)
            # Calculate the array of the variances of the lagged differences
            tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]
            # Use a linear fit to estimate the Hurst Exponent
            poly = polyfit(log(lags), log(tau), 1)
            # Return the Hurst exponent from the polyfit output
            return poly[0]*2.0
    
        i = 1
        px_ser = self.data.exo
        
        #print(px_ser.size)
        rolling_hurst = px_ser.dropna().rolling(hurst_window).apply(lambda x: hurst(x))
        
        if rules_index == 0:
            entry_rule = (rolling_hurst <= hurst_value)

            exit_rule = (rolling_hurst < rolling_hurst.rolling(hurst_median_window).median())
            return entry_rule, exit_rule
        
        if rules_index == 1:
            entry_rule = (rolling_hurst >= hurst_value)

            exit_rule = (rolling_hurst > rolling_hurst.rolling(hurst_median_window).median())
            return entry_rule, exit_rule
        
        if rules_index == 2:
            entry_rule = (rolling_hurst <= hurst_value)

            exit_rule = (rolling_hurst > rolling_hurst.rolling(hurst_median_window).median())
            return entry_rule, exit_rule
        
        if rules_index == 3:
            entry_rule = (rolling_hurst >= hurst_value)

            exit_rule = (rolling_hurst < rolling_hurst.rolling(hurst_median_window).median())
            return entry_rule, exit_rule
    
    def calculate(self, params=None, save_info=False):
    #
    #
    #  Params is a tripple like (50, 10, 15), where:
    #   50 - slow MA period
    #   10 - fast MA period
    #   15 - median period
    #
    #  On every iteration of swarming algorithm, parameter set will be different.
    #  For more information look inside: /notebooks/tmp/Swarming engine research.ipynb
    #

        if params is None:
            # Return default parameters
            direction, hurst_window, hurst_value, hurst_median_window, rules_index = self.default_opts()
        else:
            # Unpacking optimization params
            #  in order in self.opts definition
            direction, hurst_window, hurst_value, hurst_median_window, rules_index = params

        # Defining EXO price
        px = self.data.exo
                
        entry_rule, exit_rule = self.calc_entryexit_rules(hurst_window, hurst_value, hurst_median_window, rules_index)

        # Swarm_member_name must be *unique* for every swarm member
        # We use params values for uniqueness
        swarm_member_name = self.get_member_name(params)

        #
        # Calculation info
        #
        calc_info = None
        if save_info:
            calc_info = {'trailing_stop': trailing_stop}

        return swarm_member_name, entry_rule, exit_rule, calc_info

## Script settings

In [ ]:
STRATEGY_CONTEXT = {
    'strategy': { 
        'class': Strategy_RollingHurstExp,
        'exo_name': 'NG_SmartEXO_Ichi_Put_Spread_150Delta_Bi',        # <---- Select and paste EXO name from cell above
        'exo_storage': storage,          
        'opt_params': [
                        #OptParam(name, default_value, min_value, max_value, step)
                        OptParamArray('Direction', [1]),
                        #OptParam('Pct rank value', 9, 0, 1, 0.2),
                        OptParam('Roll. Hurst window', 2, 30, 60, 5),
                        OptParam('Hurst value', 0, 0.51, 0.91, 0.11),
                        OptParam('Hurst median window', 2, 25, 25, 10),
                        OptParamArray('Rules index', np.arange(1)),
                        
            ],
    },
    'swarm': {
        'members_count': 1,
        'ranking_class': RankerBestWithCorrel(window_size=-1, correl_threshold=0.5),
        'rebalance_time_function': SwarmRebalance.every_friday,

    },
    'costs':{
        'manager': CostsManagerEXOFixed,
        'context': {
            'costs_options': 3.0,
            'costs_futures': 3.0,
        }
    }
}

# Backtest class based strategy

In [ ]:
smgr = Swarm(STRATEGY_CONTEXT)
smgr.run_swarm()
smgr.pick()

# Saving results to swarms directory
smgr.save('./swarms/')

### WARNING! Loading swarm from file (don't run next cell if you want new swarm instance)

In [ ]:
smgr.strategy.data.exo.plot()

In [ ]:
figsize(10,10)
smgr.picked_equity.plot(label='Picked swarm equity');
smgr.raw_equity.plot(label='Average swarm equity');
legend(loc=2);

In [ ]:
smgr.raw_swarm.plot(legend=True)
(smgr.strategy.data.exo*-1).plot(linewidth=4, color='red')
legend(loc=2);

In [ ]:
smgr.picked_swarm.plot()

smgr.picked_swarm.sum(1).plot(label='smgr.picked_swarm-sum')
smgr.picked_equity.plot(label='Picked swarm equity')

# Swarm exposure

In [ ]:
smgr.picked_exposure.sum(axis=1).rolling(10).mean().plot()

# Swarm statistics

#### Non-picked swarm stats

In [ ]:
smgr.picked_stats

## Exo information

In [ ]:
smgr.strategy.exoinfo.exo_info

In [ ]:
smgr.strategy.exoinfo.data.exo.plot()

### Global filter information (obsolete)


## Costs information (per 1-exo unit)

In [ ]:
figsize(10,5)
smgr.strategy.costs.plot()

## Margin graphs

### EXO Margin (per 1 EXO unit)

In [ ]:
smgr.strategy.exoinfo.margin().plot()

# Saving results

In [ ]:
smgr.save('./swarms/')

In [ ]:
smgr.raw_swarm#.plot()